In [1]:
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import re

In [2]:
ebay_data = pd.read_pickle('ebay_data.pkl')
ebay_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
condition         9970 non-null object
manufacturer      6733 non-null object
memory_size       8778 non-null object
memory_type       7157 non-null object
sold_date_time    7586 non-null datetime64[ns]
sold_price        9886 non-null float64
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 468.8+ KB


In [3]:
ebay_data.head()

,condition,manufacturer,memory_size,memory_type,sold_date_time,sold_price
0,Used,None,1GB,None,2018-07-18 23:38:00,50.00
1,New,NVIDIA,2GB,DDR5,2018-07-18 23:23:00,44.61
2,New other (see details),ATI,256MB,None,2018-07-18 23:17:00,34.99
3,New,None,4GB,GDDR5,2018-07-18 22:50:00,37.00
4,Used,None,4GB,GDDR5,2018-07-18 22:44:00,125.00


In [4]:
ebay_data.describe()

,sold_price
count,9886.000000
mean,172.730563
std,185.473235
min,0.010000
25%,36.000000
50%,99.990000
75%,239.990000
max,999.990000


In [5]:
#drop rows that have all NAs
ebay_data.dropna(how="all", inplace=True)
len(ebay_data)

9971

In [6]:
#checkout target column
len(ebay_data[ebay_data['sold_price'].isnull()])

85

In [7]:
#drop rows with missing target values
ebay_data.dropna(subset=['sold_price'], inplace=True)
len(ebay_data[ebay_data['sold_price'].isnull()])

0

In [8]:
#how many rows do we have left?
len(ebay_data)

9886

In [9]:
#check how many rows are missing sold_date_time
len(ebay_data[ebay_data['sold_date_time'].isnull()])

2356

In [10]:
#drop rows that are missing sold_date_time
ebay_data.dropna(subset=['sold_date_time'], inplace=True)
len(ebay_data[ebay_data['sold_date_time'].isnull()])

0

In [11]:
#how many rows do we have left?
len(ebay_data)

7530

In [12]:
#checjout memory_type
ebay_data['memory_type'].unique()

array([None, 'DDR5', 'GDDR5', 'GDDR3', 'GDDR3 SDRAM', 'GDDR4', 'GDDR5X',
       'DDR3', 'GDDR5 SDRAM', 'SGRAM', 'DDR2', 'DDR SDRAM', 'DDR5 SDRAM',
       '64bit', 'GDDR2', 'HBM2', '6GB', 'DDR', 'HBM', 'DDR1', 'GDDR1',
       'DDR3 SDRAM', 'SDR SDRAM', 'VRAM', 'DDR5X', 'GDDR5X SDRAM',
       'DDR2 SDRAM', '64MB', '1GB', 'GDDR2 SDRAM', 'OLD', '4GB', 'Simm',
       'Ddr3', '256', 'DDR1 SDRAM', 'Gddr3', 'WRAM', 'GP107', 'NON',
       'DDR4', 'EDO RAM', 'DRAM', 'GDDR4 SDRAM', 'N/A', 'SDRAM', '3D-RAM',
       '8 GB GDDR5'], dtype=object)

In [13]:
ebay_data['memory_type'].value_counts()

GDDR5           3139
GDDR5X           550
DDR3             535
DDR5             431
GDDR3            206
GDDR5 SDRAM      180
DDR2             136
HBM2              42
DDR SDRAM         30
GDDR3 SDRAM       28
GDDR2             26
DDR3 SDRAM        24
VRAM              16
HBM               14
DDR1              12
GP107             12
DDR               11
DDR2 SDRAM        10
SDR SDRAM         10
DDR5 SDRAM         9
GDDR4              7
DDR5X              7
GDDR5X SDRAM       4
GDDR1              3
DDR1 SDRAM         3
SGRAM              3
DDR4               3
64bit              2
DRAM               2
6GB                2
WRAM               1
OLD                1
Ddr3               1
8 GB GDDR5         1
Simm               1
3D-RAM             1
N/A                1
64MB               1
EDO RAM            1
1GB                1
256                1
NON                1
SDRAM              1
GDDR4 SDRAM        1
GDDR2 SDRAM        1
Gddr3              1
4GB                1
Name: memory_

In [14]:
#clean up memory type column
#ebay_data['memory_type'] = ebay_data['memory_type'].str.replace('DDR?\d?X (\w)','')
ebay_data['memory_type'].unique()

array([None, 'DDR5', 'GDDR5', 'GDDR3', 'GDDR3 SDRAM', 'GDDR4', 'GDDR5X',
       'DDR3', 'GDDR5 SDRAM', 'SGRAM', 'DDR2', 'DDR SDRAM', 'DDR5 SDRAM',
       '64bit', 'GDDR2', 'HBM2', '6GB', 'DDR', 'HBM', 'DDR1', 'GDDR1',
       'DDR3 SDRAM', 'SDR SDRAM', 'VRAM', 'DDR5X', 'GDDR5X SDRAM',
       'DDR2 SDRAM', '64MB', '1GB', 'GDDR2 SDRAM', 'OLD', '4GB', 'Simm',
       'Ddr3', '256', 'DDR1 SDRAM', 'Gddr3', 'WRAM', 'GP107', 'NON',
       'DDR4', 'EDO RAM', 'DRAM', 'GDDR4 SDRAM', 'N/A', 'SDRAM', '3D-RAM',
       '8 GB GDDR5'], dtype=object)

In [15]:
 #check out manufacturer column
ebay_data['manufacturer'].unique()

array([None, 'NVIDIA', 'ATI', 'AMD', '3DFX', 'Yes', 'EVGA', 'Gigabyte',
       'EASYDIY', 'Fstop Labs', 'ZOTAC', 'Asus', 'Matrox', 'For ATI',
       'AMD/ATi', 'Radeon', 'nVIDIA', 'XFX', 'S3', 'No', 'Other', 'MSI',
       'Dell Nvidia NVS 310', 'EVGA GeForce', 'ASUS',
       'Sapphire Technology', '650MHz', 'For NVIDIA GeForce', 'For AMD',
       'Trident', 'Nvidia', 'MSI Computer', 'StarTech', '3Dfx Voodoo2',
       'Geforce GT210', 'AMD/ATI', 'VideoChip', 'manufacturer', 'Evga',
       'Apple', 'NVIDIA GT 10 Series', 'Gemini', 'Nvidia/MSI', 'SiS',
       'PNY', 'for NVIDIA', 'AMD ATI', 'Dell/Nvidia', 'Cirrus Logic',
       'Intel', 'MSI COMPUTER', 'Barco', 'asus', 'EVGA]', '3dfx',
       'XtremeMac', 'NVIDIA Quadro 4000', 'AMD FirePro W5100', 'Voodoo',
       'PowerColor', 'AMD Radeon RX 580', 'DDR5', '825MHz', '3DLabs',
       'cirrus', 'Sapphire Tech', '3 Years', 'NONE', 'AMD Radeon HD 7470',
       '3Dlabs', 'Oak Technology', 'SAPPHIRE', 'Gigabyte Technology',
       'Zotac', 'nVi

In [16]:
ebay_data['manufacturer'] = ebay_data['manufacturer'].str.upper()
ebay_data['manufacturer'].unique()
#drop manufacturer = 'YES' ? What to do about AMD / NVIDIA?

array([None, 'NVIDIA', 'ATI', 'AMD', '3DFX', 'YES', 'EVGA', 'GIGABYTE',
       'EASYDIY', 'FSTOP LABS', 'ZOTAC', 'ASUS', 'MATROX', 'FOR ATI',
       'AMD/ATI', 'RADEON', 'XFX', 'S3', 'NO', 'OTHER', 'MSI',
       'DELL NVIDIA NVS 310', 'EVGA GEFORCE', 'SAPPHIRE TECHNOLOGY',
       '650MHZ', 'FOR NVIDIA GEFORCE', 'FOR AMD', 'TRIDENT',
       'MSI COMPUTER', 'STARTECH', '3DFX VOODOO2', 'GEFORCE GT210',
       'VIDEOCHIP', 'MANUFACTURER', 'APPLE', 'NVIDIA GT 10 SERIES',
       'GEMINI', 'NVIDIA/MSI', 'SIS', 'PNY', 'FOR NVIDIA', 'AMD ATI',
       'DELL/NVIDIA', 'CIRRUS LOGIC', 'INTEL', 'BARCO', 'EVGA]',
       'XTREMEMAC', 'NVIDIA QUADRO 4000', 'AMD FIREPRO W5100', 'VOODOO',
       'POWERCOLOR', 'AMD RADEON RX 580', 'DDR5', '825MHZ', '3DLABS',
       'CIRRUS', 'SAPPHIRE TECH', '3 YEARS', 'NONE', 'AMD RADEON HD 7470',
       'OAK TECHNOLOGY', 'SAPPHIRE', 'GIGABYTE TECHNOLOGY',
       'NVIDIA QUADRO NVS 420', 'ATI / AMD', 'PC CHIPS', 'VISIONTEK',
       'AMD RX SERIES', 'NVIDIA GTX 10 SERIES'

In [17]:
ebay_data['manufacturer'].value_counts()

NVIDIA                   3267
AMD                      1328
ATI                       378
YES                        30
EVGA                       22
ASUS                       17
NO                         13
MATROX                     13
GIGABYTE                   12
MSI                        11
FOR NVIDIA GEFORCE         10
VIDEOCHIP                   8
OTHER                       7
POWERCOLOR                  6
3DFX                        5
MSI COMPUTER                5
ZOTAC                       4
XFX                         4
3 YEARS                     3
FSTOP LABS                  3
RADEON                      3
AMD ATI                     3
CIRRUS LOGIC                3
AMD/ATI                     3
S3                          3
SIS                         3
NVIDIA QUADRO 4000          2
NVIDIA GT 10 SERIES         2
3DLABS                      2
EASYDIY                     2
                         ... 
GEFORCE GT210               1
FOR AMD                     1
DELL NVIDI

In [18]:
#ebay_data[(ebay_data['manufacturer'].str.find('NVIDIA') != -1 )&(ebay_data['manufacturer'].notnull())]['manufacturer'] = "NVIDIA"

In [19]:
#ebay_data[ebay_data['manufacturer'].str.find('NVIDIA') != -1] = "NVIDIA"
ebay_data['manufacturer'].value_counts()

NVIDIA                   3267
AMD                      1328
ATI                       378
YES                        30
EVGA                       22
ASUS                       17
NO                         13
MATROX                     13
GIGABYTE                   12
MSI                        11
FOR NVIDIA GEFORCE         10
VIDEOCHIP                   8
OTHER                       7
POWERCOLOR                  6
3DFX                        5
MSI COMPUTER                5
ZOTAC                       4
XFX                         4
3 YEARS                     3
FSTOP LABS                  3
RADEON                      3
AMD ATI                     3
CIRRUS LOGIC                3
AMD/ATI                     3
S3                          3
SIS                         3
NVIDIA QUADRO 4000          2
NVIDIA GT 10 SERIES         2
3DLABS                      2
EASYDIY                     2
                         ... 
GEFORCE GT210               1
FOR AMD                     1
DELL NVIDI

In [20]:
#clean up memory size column (make it numeric by converting all to KB)
ebay_data['memory_size'].unique()

array(['1GB', '2GB', '256MB', '4GB', '512 MB', '8GB', None, '3GB',
       '2.5GB', '11GB', '6GB', '128MB', '16MB', '8', '3 GB', '512MB',
       '1.5GB', '8MB', '2 GB', '64Kb', '5GB', '16GB', '12GB', '3072MB',
       '896MB', '1 GB', '768MB', '768 MB', '64MB', '4 MB', '1024MB',
       '1MB', '16 GB', '4 GB', '8GB Memory', '32MB', '2G', '6 GB', '2MB',
       '2GB 256-Bit GDDR5', '4096MB', '12 GB', '2048MB', '512Kb', '12MB',
       '6144MB', '4MB', 'Rage XL 8M', '1.25GB', '218MB', '1280MB',
       'Description', '1536MB', '4096M (4GB)', '640MB', '256kb', '24GB',
       '256MB Onboard Supporting 512MB', '8 GB', '1G TC 4G', '1 MB',
       '2.5 GB', '320MB', '32 MB', '1.75GB', '4GB - 1GB', '1.28GB',
       '256KB', '256GB', '2048 MB', '00FC903', '32GB', '128 MB', '256 MB',
       '1792MB', '8192MB', '1024m', '3mb', '1024 MB', '384MB', '4',
       'SEE PICS', '8GB GDDR5'], dtype=object)

In [21]:
ebay_data['memory_size'].value_counts(dropna=False)

8GB                               1412
4GB                               1317
2GB                                954
1GB                                852
NaN                                831
6GB                                401
3GB                                387
11GB                               325
512MB                              301
256MB                              237
128MB                               90
12GB                                35
64MB                                34
1.5GB                               31
8MB                                 23
768MB                               20
32MB                                18
16GB                                17
16MB                                16
2 GB                                16
5GB                                 14
2.5GB                               13
4 GB                                13
896MB                               11
1 GB                                11
4MB                      

In [22]:
#how many Nones do we have?
len(ebay_data[ebay_data['memory_size'].isnull()])

831

In [23]:
def convert_to_kb(mem_string):
    try:
        mem_string = mem_string.upper()
        amount_type_split = re.match(r"([0-9]+(\.[0-9]+)?)([A-Z]B)", mem_string, re.I)
        amount = float(amount_type_split.group(1))
        mem_type = amount_type_split.group(3).upper()
        #print(amount, mem_type)
        if mem_type == 'GB':
            amount = amount *(1024**2)
        elif mem_type == 'MB':
            amount = amount * 1024
        #print(amount)
        return amount
    except:
        #print('Exception!')
        return None
ebay_data['memory_size_kb'] = ebay_data['memory_size'].apply(convert_to_kb)
ebay_data.drop(columns=['memory_size'], inplace=True)


In [24]:
#check if it works
convert_to_kb('16GB')

16777216.0

In [25]:
#find empty values in memory_size
len(ebay_data[ebay_data['memory_size_kb'].isnull()])

930

In [26]:
#replace empty values with the mean
ebay_data['memory_size_kb'].fillna(ebay_data['memory_size_kb'].mean(), inplace=True)

In [27]:
#check out condition column
ebay_data['condition'].value_counts()

Used                        5308
New                         1062
New other (see details)      545
For parts or not working     295
Seller refurbished           237
Manufacturer refurbished      82
Name: condition, dtype: int64

In [28]:
ebay_data_small = ebay_data[['memory_size_kb','sold_price']]

In [29]:
#round ebay sold_date_time to nearest day to merge with crypto_data and rename the column
ebay_data['sold_date'] = ebay_data['sold_date_time'].dt.round('d')
ebay_data.drop(columns=['sold_date_time'], inplace=True)

In [30]:
#figure out ebay data date_range
ebay_data['sold_date'].describe()

count                    7530
unique                     46
top       2018-06-07 00:00:00
freq                      332
first     2018-06-04 00:00:00
last      2018-07-19 00:00:00
Name: sold_date, dtype: object

In [31]:
#we have june 4th to july 19th!

In [32]:
crypto_data = pd.read_csv('bit_coin_2018-06-03_2018-07-18.csv')
#crypto_data['Date'] = pd.to_datetime(crypto_data['Date'])
crypto_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 2 columns):
Date           51 non-null object
Close Price    49 non-null float64
dtypes: float64(1), object(1)
memory usage: 896.0+ bytes


In [33]:
crypto_data.head()

,Date,Close Price
0,2018-06-01 00:00:00,7518.24
1,2018-06-02 00:00:00,7636.19
2,2018-06-03 00:00:00,7711.36
3,2018-06-04 00:00:00,7490.59
4,2018-06-05 00:00:00,7616.89


In [34]:
#looks like it's broken down daily closing price, let's rename the column for the merge
crypto_data.rename(columns={'Close Price':'bit_coin_close_price'}, inplace=True)

In [35]:
#check for weirdness
crypto_data.tail()

,Date,bit_coin_close_price
46,2018-07-17 00:00:00,7314.94
47,2018-07-18 00:00:00,7378.76
48,2018-07-19 20:30:00,7460.48
49,This data was produced from the CoinDesk price...,NaN
50,http://www.coindesk.com/price/,NaN


In [36]:
#drop the null rows
crypto_data.dropna(inplace=True)
crypto_data.tail()

,Date,bit_coin_close_price
44,2018-07-15 00:00:00,6349.04
45,2018-07-16 00:00:00,6726.40
46,2018-07-17 00:00:00,7314.94
47,2018-07-18 00:00:00,7378.76
48,2018-07-19 20:30:00,7460.48


In [37]:
crypto_data['Date'] = pd.to_datetime(crypto_data['Date'], infer_datetime_format=True).dt.round('d')

In [38]:
crypto_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49 entries, 0 to 48
Data columns (total 2 columns):
Date                    49 non-null datetime64[ns]
bit_coin_close_price    49 non-null float64
dtypes: datetime64[ns](1), float64(1)
memory usage: 1.1 KB


In [39]:
#look for inconsistencies in the data
crypto_data['Date'].describe()

count                      49
unique                     49
top       2018-06-11 00:00:00
freq                        1
first     2018-06-01 00:00:00
last      2018-07-20 00:00:00
Name: Date, dtype: object

In [40]:
#how many rows do we have left in ebay data? don't want to lose any in the merge
len(ebay_data)

7530

In [41]:
ebay_data['sold_date'].describe()

count                    7530
unique                     46
top       2018-06-07 00:00:00
freq                      332
first     2018-06-04 00:00:00
last      2018-07-19 00:00:00
Name: sold_date, dtype: object

In [42]:
#merge in crypto_data
data = pd.merge(ebay_data,crypto_data, left_on="sold_date", right_on='Date')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7406 entries, 0 to 7405
Data columns (total 8 columns):
condition               7405 non-null object
manufacturer            5136 non-null object
memory_type             5388 non-null object
sold_price              7406 non-null float64
memory_size_kb          7406 non-null float64
sold_date               7406 non-null datetime64[ns]
Date                    7406 non-null datetime64[ns]
bit_coin_close_price    7406 non-null float64
dtypes: datetime64[ns](2), float64(3), object(3)
memory usage: 520.7+ KB


In [43]:
data.head()

,condition,manufacturer,memory_type,sold_price,memory_size_kb,sold_date,Date,bit_coin_close_price
0,Used,NVIDIA,DDR3,40.99,2.097152e+06,2018-07-18,2018-07-18,7378.76
1,Seller refurbished,NVIDIA,GDDR5,59.99,1.048576e+06,2018-07-18,2018-07-18,7378.76
2,Used,NVIDIA,GDDR5,59.00,1.048576e+06,2018-07-18,2018-07-18,7378.76
3,Used,ATI,DDR2,7.00,1.310720e+05,2018-07-18,2018-07-18,7378.76
4,Seller refurbished,AMD,DDR5,260.00,4.488604e+06,2018-07-18,2018-07-18,7378.76


In [44]:
data.drop(columns=['Date'], inplace=True)

In [45]:
data.head()

,condition,manufacturer,memory_type,sold_price,memory_size_kb,sold_date,bit_coin_close_price
0,Used,NVIDIA,DDR3,40.99,2.097152e+06,2018-07-18,7378.76
1,Seller refurbished,NVIDIA,GDDR5,59.99,1.048576e+06,2018-07-18,7378.76
2,Used,NVIDIA,GDDR5,59.00,1.048576e+06,2018-07-18,7378.76
3,Used,ATI,DDR2,7.00,1.310720e+05,2018-07-18,7378.76
4,Seller refurbished,AMD,DDR5,260.00,4.488604e+06,2018-07-18,7378.76


In [46]:
#data_small = data[['sold_price', 'memory_size_kb', 'Close Price']]

In [47]:
#sns.pairplot(data_small)

In [48]:
data.to_pickle('graphics_card_data.pkl')